In [4]:
"""
use pathlib to load items.json. separately, we want to load all the jsons in jei/*.json into an array. The jei folder contains the recipe json of the categories, and the items.json has a field with the tooltip. We will in the next steps extract the EMC value from the tooltip to create a lookup table and then we will use the recipe jsons to try and find a EMC loop that produces more EMC than it consumes
"""

import json
from pathlib import Path
import polars as pl

# Load items.json
items_path = Path("items.json")
items_data = pl.read_json(items_path)
print(f"Loaded {len(items_data)} items from {items_path}")

Loaded 38680 items from items.json


In [5]:
# Load all jsons in jei/*.json into an array
from tqdm import tqdm
jei_folder = Path("jei")
jei_jsons = []

for json_file in tqdm(list(jei_folder.glob("*.json"))):
    with open(json_file) as f:
        jei_jsons.extend(json.load(f))
print(f"Loaded {len(jei_jsons)} recipe jsons from {jei_folder}")

100%|██████████| 288/288 [00:25<00:00, 11.32it/s]

Loaded 125584 recipe jsons from jei


In [6]:
recipes = pl.DataFrame(jei_jsons)

items_data = items_data.with_column(
    pl.col("tooltip").apply(get_emc_from_tooltip).alias("emc")
)

KeyboardInterrupt: 

In [ ]:
# Define the get_emc_from_tooltip() function
def get_emc_string_from_tooltip(tooltip: str) -> str | None:
    for line in tooltip.split("\n"):
        #  EMC: 32 (✗)
        if line.startswith("EMC: "):
            emc = line.split(":")[1].replace(" (✗)", "")
            return emc
    return None


# Test the function with the provided test_tooltip
test_tooltip = items_data[0]["tooltip"]
print(test_tooltip)
test_emc = get_emc_string_from_tooltip(test_tooltip)
print(test_emc, type(test_emc))

In [ ]:
seen = set()
for item in items_data:
    emc_string = get_emc_string_from_tooltip(item["tooltip"])
    if emc_string is not None:
        if not emc_string in seen:
            seen.add(emc_string)
            print(emc_string)

In [ ]:
seen_words = set()
import re
for x in seen:
    for word in x.split(" "):
        if re.match(r"^[\d,\.]+$", word):
            continue
        seen_words.add(word)
print(seen_words)

In [ ]:
number_words = {
    "Million": 1_000_000,
    "Billion": 1_000_000_000,
    "Trillion": 1_000_000_000_000,
    "Quadrillion": 1_000_000_000_000_000,
    "Quintillion": 1_000_000_000_000_000_000,
}


In [ ]:
def get_emc_value_from_string(emc_string: str) -> float:
    emc_string = emc_string.replace(",", "")
    for word, value in number_words.items():
        if word in emc_string:
            return value * float(emc_string.replace(word, ""))
    return float(emc_string)
print(get_emc_value_from_string(test_emc))
print(get_emc_value_from_string("1,000.3 Billion"))

In [ ]:
for item in items_data:
    try:
        emc_string = get_emc_string_from_tooltip(item["tooltip"])
        if emc_string is not None:
            item["emc"] = get_emc_value_from_string(emc_string)
        else:
            item["emc"] = None
    except Exception as e:
        item["emc"] = None
        print(f"Failed to get emc for {item['id']} with emc string '{emc_string}' ({type(emc_string)})", e)

In [ ]:
sorted_values = sorted([(item["emc"], item["id"]) for item in items_data if item["emc"] is not None], reverse=True)

# Print the top 3 and bottom 3 non-None values
print("Top 3 non-None values:")
print(sorted_values[:3])
print("Bottom 3 non-None values:")
print(sorted_values[-3:])

In [ ]:
jei_jsons[0]

In [ ]:
# [x for x in jei_jsons if x["categoryTitle"] == "Crafting" and any(True for i in x["ingredients"] if i["role"] == "INPUT" and i["ingredientId"] == "minecraft:blaze_rods")]
for recipe in jei_jsons:
    if recipe["categoryTitle"] == "Crafting":
        for ingredient in recipe["ingredients"]:
            if ingredient["role"] == "INPUT": # and ingredient["ingredientId"] == "minecraft:blaze_rods":
                print(recipe)
                raise "stop"